In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time


resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="")
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices("TPU"))

strategy = tf.distribute.TPUStrategy(resolver)


INFO:tensorflow:Initializing the TPU system: grpc://10.28.18.58:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.28.18.58:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                 if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
# Loads data from a file
def load_data(path):
  input_file = os.path.join(path)

  # Open the file read-only
  with open(input_file, "r") as f:
    # Convert file contents to ascii
    data = unicode_to_ascii(f.read())
  return data.split('\n')

# Load sentence pairs
normal_sentences = load_data("drive/MyDrive/normal.training.txt")
simple_sentences = load_data("drive/MyDrive/simple.training.txt")

# Preprocess sentences pairs
normal_sentences = [preprocess_sentence(sentence) for sentence in normal_sentences if sentence != ""]
simple_sentences = [preprocess_sentence(sentence) for sentence in simple_sentences if sentence != ""]

sentence_amount = 15000
normal_sentences = normal_sentences[:sentence_amount]
simple_sentences = simple_sentences[:sentence_amount]


In [ ]:
print(normal_sentences[-1])
print(simple_sentences[-1])

import numpy as np
# Import glove embeddings
embeddings_index = {}
embeddingFile = open("drive/MyDrive/glove.6B.300d.txt", encoding="utf-8")
for line in embeddingFile:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
embeddingFile.close()
print("Number of words in glove embeddings = ", len(embeddings_index))

<start> moving ice has formed depressions , which serve as basins for numerous glacial lakes . <end>
<start> moving ice has formed depressions , which serve as basins for numerous glacial lakes . <end>
Number of words in glove embeddings =  400000


In [ ]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token="OOV")
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
  return tensor, lang_tokenizer

In [ ]:
def load_dataset(targ_lang, inp_lang, num_examples=None):
  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

with strategy.scope():
  num_examples = 15000
  input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(normal_sentences, simple_sentences, num_examples)

  words = len(inp_lang.word_index)

  # + 1 to compensate for oov token
  embedding_matrix = np.zeros((words + 1, 300))

  print(inp_lang.word_index.items())

  for word, i in inp_lang.word_index.items():
      if i < words:
          embedding_vector = embeddings_index.get(word)
          if embedding_vector is not None:
              embedding_matrix[i] = embedding_vector

  # Calculate max_length of the target tensors
  max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

  # Creating training and validation sets using an 90-10 split
  input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

  # Show length
  print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

dict_items([('OOV', 1), ('the', 2), ('.', 3), ('<start>', 4), ('<end>', 5), (',', 6), ('of', 7), ('in', 8), ('a', 9), ('and', 10), ('is', 11), ('to', 12), ('was', 13), ('it', 14), ('for', 15), ('as', 16), ('on', 17), ('by', 18), ('that', 19), ('s', 20), ('he', 21), ('with', 22), ('are', 23), ('from', 24), ('an', 25), ('at', 26), ('or', 27), ('his', 28), ('be', 29), ('this', 30), ('which', 31), ('also', 32), ('were', 33), ('has', 34), ('one', 35), ('they', 36), ('commune', 37), ('not', 38), ('first', 39), ('france', 40), ('city', 41), ('found', 42), ('have', 43), ('region', 44), ('after', 45), ('united', 46), ('had', 47), ('can', 48), ('used', 49), ('but', 50), ('other', 51), ('its', 52), ('states', 53), ('north', 54), ('department', 55), ('when', 56), ('people', 57), ('their', 58), ('called', 59), ('two', 60), ('there', 61), ('football', 62), ('who', 63), ('about', 64), ('made', 65), ('known', 66), ('most', 67), ('new', 68), ('many', 69), ('she', 70), ('de', 71), ('part', 72), ('her', 

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 109]), TensorShape([64, 135]))

In [ ]:
with strategy.scope():
  class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
      super(Encoder, self).__init__()
      self.batch_sz = batch_sz
      self.enc_units = enc_units
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix])
      self.gru = tf.keras.layers.GRU(self.enc_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
      x = self.embedding(x)
      output, state = self.gru(x, initial_state=hidden)
      return output, state

    def initialize_hidden_state(self):
      return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
with strategy.scope():
  class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
      super(BahdanauAttention, self).__init__()
      self.W1 = tf.keras.layers.Dense(units)
      self.W2 = tf.keras.layers.Dense(units)
      self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
      # query hidden state shape == (batch_size, hidden size)
      # query_with_time_axis shape == (batch_size, 1, hidden size)
      # values shape == (batch_size, max_len, hidden size)
      # we are doing this to broadcast addition along the time axis to calculate the score
      query_with_time_axis = tf.expand_dims(query, 1)

      # score shape == (batch_size, max_length, 1)
      # we get 1 at the last axis because we are applying score to self.V
      # the shape of the tensor before applying self.V is (batch_size, max_length, units)
      score = self.V(tf.nn.tanh(
          self.W1(query_with_time_axis) + self.W2(values)))

      # attention_weights shape == (batch_size, max_length, 1)
      attention_weights = tf.nn.softmax(score, axis=1)

      # context_vector shape after sum == (batch_size, hidden_size)
      context_vector = attention_weights * values
      context_vector = tf.reduce_sum(context_vector, axis=1)

      return context_vector, attention_weights



In [ ]:
with strategy.scope():
  class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
      super(Decoder, self).__init__()
      self.batch_sz = batch_sz
      self.dec_units = dec_units
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
      self.gru = tf.keras.layers.GRU(self.dec_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')
      self.fc = tf.keras.layers.Dense(vocab_size)

      # used for attention
      self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
      # enc_output shape == (batch_size, max_length, hidden_size)
      context_vector, attention_weights = self.attention(hidden, enc_output)

      # x shape after passing through embedding == (batch_size, 1, embedding_dim)
      x = self.embedding(x)

      # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
      x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

      # passing the concatenated vector to the GRU
      output, state = self.gru(x)

      # output shape == (batch_size * 1, hidden_size)
      output = tf.reshape(output, (-1, output.shape[2]))

      # output shape == (batch_size, vocab)
      x = self.fc(output)

      return x, state, attention_weights

In [ ]:
with strategy.scope():
  encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

  sample_hidden = encoder.initialize_hidden_state()
  sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

  attention_layer = BahdanauAttention(10)
  attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

  print("Attention result shape: (batch size, units)", attention_result.shape)
  print("Attention weights shape: (batch_size, sequence_length, 1)", attention_weights.shape)

  decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

  sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)
  print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)


  optimizer = tf.keras.optimizers.Adam()
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
  
  checkpoint_dir = './drive/MyDrive/modelCheckpoints'
  checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
  checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)


Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 109, 1)
Decoder output shape: (batch_size, vocab size) (64, 31796)


In [ ]:



def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)


In [ ]:
with strategy.scope():
  @tf.function
  def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
      enc_output, enc_hidden = encoder(inp, enc_hidden)

      dec_hidden = enc_hidden

      dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

      # Teacher forcing - feeding the target as the next input
      for t in range(1, targ.shape[1]):
        # passing enc_output to the decoder
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

        loss += loss_function(targ[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss



In [ ]:
print("Started")

with strategy.scope():
  EPOCHS = 30

  for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
      batch_loss = strategy.run(train_step, args=(inp, targ, enc_hidden))
      total_loss += batch_loss.values[0].numpy()
      if batch % 100 == 0:
        print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.values[0].numpy():.4f}')
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix=checkpoint_prefix)
    print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')


Started
Epoch 1 Batch 0 Loss 1.9426
Epoch 1 Batch 100 Loss 1.4014
Epoch 1 Batch 200 Loss 1.3132
Epoch 1 Loss 1.2647
Time taken for 1 epoch 361.01 sec

Epoch 2 Batch 0 Loss 1.2017
Epoch 2 Batch 100 Loss 1.1312
Epoch 2 Batch 200 Loss 0.9523
Epoch 2 Loss 1.1425
Time taken for 1 epoch 152.96 sec

Epoch 3 Batch 0 Loss 1.0474
Epoch 3 Batch 100 Loss 1.1651
Epoch 3 Batch 200 Loss 1.1051
Epoch 3 Loss 1.0795
Time taken for 1 epoch 152.99 sec

Epoch 4 Batch 0 Loss 1.0708
Epoch 4 Batch 100 Loss 1.1261
Epoch 4 Batch 200 Loss 1.0410
Epoch 4 Loss 1.0282
Time taken for 1 epoch 153.00 sec

Epoch 5 Batch 0 Loss 0.8885
Epoch 5 Batch 100 Loss 1.0467
Epoch 5 Batch 200 Loss 0.8730
Epoch 5 Loss 0.9631
Time taken for 1 epoch 153.01 sec

Epoch 6 Batch 0 Loss 1.0411
Epoch 6 Batch 100 Loss 0.9389
Epoch 6 Batch 200 Loss 1.0158
Epoch 6 Loss 0.8847
Time taken for 1 epoch 153.02 sec

Epoch 7 Batch 0 Loss 0.8706
Epoch 7 Batch 100 Loss 0.7761
Epoch 7 Batch 200 Loss 0.7256
Epoch 7 Loss 0.7925
Time taken for 1 epoch 153

In [ ]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = inp_lang.texts_to_sequences(sentence.split(' '))[0]

  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()


In [ ]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  #print('Input:', sentence)
  #print('Predicted translation:', result)

  words = result.split(" ")
  idxToRemove = []
  for idx, word in enumerate(words):
    if idx != 0:
      if word == words[idx - 1]:
        idxToRemove.append(idx)
  for idx in reversed(idxToRemove):
    del words[idx]
  return " ".join(words)

idx = 0
print(normal_sentences[idx])
print(translate(normal_sentences[idx]))



<start> when green tore his acl in a preseason game , warner took over as the rams tentative starter . <end>
courcite is a strong round of the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the most important to their autonomous community by the 


In [ ]:
local_device_option = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")
checkpoint.save(file_prefix=checkpoint_prefix, options=local_device_option)

'./drive/MyDrive/modelCheckpoints/ckpt-5'

In [ ]:
with open('drive/MyDrive/input_tokenizer5.pickle', 'wb') as handle:
    pickle.dump(input_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)